### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this code , you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [19]:
!pip install langchain

In [20]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

# import openai
# openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [21]:
!pip install langchain_groq

In [22]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000014CD4AC65D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000014CD4AC7CE0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [23]:
!pip install langchain_core

In [24]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [25]:
result

AIMessage(content='Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal option)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is used with friends and family)\n* **Coucou, ça va ?** (This is a very informal, friendly greeting)\n\n\n\nLet me know if you\'d like to learn more French phrases!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 21, 'total_tokens': 122, 'completion_time': 0.183636364, 'prompt_time': 0.002138226, 'queue_time': 0.257685024, 'total_time': 0.18577459}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--bd1e5e05-599e-468e-af48-79799e700445-0', usage_metadata={'input_tokens': 21, 'output_tokens': 101, 'total_tokens': 122})

In [26]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal option)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is used with friends and family)\n* **Coucou, ça va ?** (This is a very informal, friendly greeting)\n\n\n\nLet me know if you\'d like to learn more French phrases!\n'

In [27]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'The most common way to say "Hello How are you?" in French is:\n\n**Bonjour, comment allez-vous ?**\n\nHere\'s a breakdown:\n\n* **Bonjour** means "Hello"\n* **Comment** means "How"\n* **allez-vous** is the formal way to ask "are you?"\n\n\nLet me know if you\'d like to know other ways to say this, depending on the formality of the situation!\n'

In [28]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [29]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [30]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [31]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [32]:
!pip install streamlit